In [ ]:
import pandas as pd

# Step 1: Load Raw Survey Data
raw_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\0_South_Africa_Wildland_Fire_Survey_Summary_Sheet.xlsx'
cleaned_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\1_Replicated_Cleaned_South_Africa_Wildland_Fire_Survey.xlsx'

# Load the raw survey data
raw_data = pd.read_excel(raw_file_path)

# Step 2: Clean the Data
# Remove irrelevant columns, for example, "IP Address", "Email Address", etc.
columns_to_remove = ['IP Address', 'Email Address', 'First Name', 'Last Name', 'Custom Data 1']
cleaned_data = raw_data.drop(columns=columns_to_remove, errors='ignore')

# Handle missing data (optional: depending on your need, you might drop NaN or fill them with a value)
cleaned_data = cleaned_data.fillna('')  # or cleaned_data.dropna() to remove rows with missing values

# Save the cleaned file
cleaned_data.to_excel(cleaned_file_path, index=False)

print("Step 1 completed: Data cleaned and saved to:", cleaned_file_path)


In [ ]:
import pandas as pd

# Step 2: Load Cleaned Data (from Step 1)
cleaned_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\0_South_Africa_Wildland_Fire_Survey_Summary_Sheet.xlsx'
categorized_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\2_Final_Categorized_South_Africa_Wildland_Fire_Survey.xlsx'

# Load the cleaned data
cleaned_data = pd.read_excel(cleaned_file_path)

# Define explicit question-to-category mapping based on the 5 Rs and other categories
question_category_mapping = {
    "Response": [
        "Formal Incident Command System", "What kind of equipment does your organization use",
        "Firefighters are equipped with appropriate tools", 
        "What kind of equipment does your community have to manage wildfires",
        "Firefighters are equipped with appropriate personal protective equipment"
    ],
    "Readiness": [
        "There is a national training program", "There are sufficient personnel to deal with wildland fires",
        "Wildland fire suppression brigades", "Wildland fire suppression brigades are professionally trained and equipped",
        "There is a national training program / certification process"
    ],
    "Recovery": [
        "There are programs to support communities", "There are restoration guidelines for affected regions",
        "There are restoration guidelines for ecosystems affected by wildland fires",
        "There are programs to support communities after a damaging wildfire"
    ],
    "Risk Reduction": [
        "What are the top two causes of wildland fires", 
        "For fires that are set intentionally", 
        "What are the top two causes of human-originated wildfires?",
        "For fires that are set intentionally, what do you think is the ignitor's intended purpose?",
        "Your community uses fuel barriers (fuel/fire breaks)"
    ],
    "Review & Analysis": [
        "What are the three most unwanted effects of wildland fires", 
        "What are the three most desirable effects of wildland fire or prescribed fire",
        "What are three aspects of wildland fire management that are most successful", 
        "What are three aspects of wildland fire management that could use improvement", 
        "What are three aspects of wildland fire management that South Africa could share", 
        "What types of fuel burn most often in wildland fires", 
        "Please identify 3 – 4 biomes that burn most frequently",
        "There is sufficient wildland fire related research in South Africa",
        "If you think there is insufficient wildland fire related research in South Africa, which topics would you suggest for further study?",
        "There are official processes for investigating the origin and cause of wildland fires",
        "There are national or regional wildland fire prevention and fire management programs"
    ],
    "Miscellaneous": [
        "Your profession – check all that apply", "How long have you been involved in fire management",
        "Who decides to use/approve the use of aircraft", 
        "There are Unmanned Aerial Systems (UAS)/drones/robots used for wildland fire management purposes",
        "Research is carried out on the ecological effects of fire in the protected areas",
        "Traditional uses of fire and perspectives of indigenous communities were considered"
    ],
    "Additional Insight": [
        "Please elaborate on anything else you want to share",
        "There is a coordinating agency or ministry for national and/or local emergencies",
        "Wildland fires are managed differently in protected natural areas",
        "Would you be interested in participating in future data gathering"
    ]
}

# Function to categorize questions based on exact match or keyword containment
def categorize_question(question, mapping):
    question_lower = question.lower()  # Normalize case
    for category, questions in mapping.items():
        if any(q.lower() in question_lower for q in questions):  # Match by containment in question
            return category
    return "Miscellaneous"

# Initialize the result DataFrame to store categorized responses
result_data = pd.DataFrame()

# Loop through each column in the cleaned data (since not all questions follow a clear naming convention)
for col in cleaned_data.columns:
    if col != 'Respondent ID':  # Avoid categorizing the respondent ID
        question_data = cleaned_data[['Respondent ID', col]].copy()
        question_data['Question'] = col  # The column name is the question
        question_data['Response'] = cleaned_data[col]
        
        # Apply the categorization based on the mapping
        question_data['Category'] = question_data['Question'].apply(lambda q: categorize_question(q, question_category_mapping))
        
        # Append this to the final result
        result_data = pd.concat([result_data, question_data], ignore_index=True)

# Drop rows where the response is empty
result_data = result_data[result_data['Response'].notna() & (result_data['Response'] != '')]

# Rearrange the columns to match the desired final structure
result_data = result_data[['Respondent ID', 'Category', 'Question', 'Response']]

# Save the final categorized file
result_data.to_excel(categorized_file_path, index=False)

print("Step 2 completed: Data categorized and saved to:", categorized_file_path)


In [4]:
import pandas as pd

# Load the cleaned and categorized data
cleaned_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\1_Replicated_Cleaned_South_Africa_Wildland_Fire_Survey.xlsx'
categorized_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\2_Final_Categorized_South_Africa_Wildland_Fire_Survey.xlsx'

cleaned_data = pd.read_excel(cleaned_file_path)
categorized_data = pd.read_excel(categorized_file_path)

# Rename columns for easier access (from cleaned data)
cleaned_data.rename(columns={
    'Your gender (optional):': 'Gender',
    'Your race (optional):\t\t': 'Race',
    'In which province do you primarily live in?': 'Province',
    'Your profession – check all that apply': 'Occupation'
}, inplace=True)

# Columns needed for demographics
demographic_columns = ['Respondent ID', 'Race', 'Gender', 'Province', 'Occupation']

# Merge datasets on 'Respondent ID'
merged_data = pd.merge(categorized_data, cleaned_data[demographic_columns], on='Respondent ID', how='left')

# Create a new 'Demography' column by concatenating the demographic fields
merged_data['Demography'] = merged_data[['Race', 'Gender', 'Province']].apply(lambda x: ' | '.join(x.dropna()), axis=1)

# Rearrange the columns to match the desired structure
final_columns = ['Respondent ID', 'Category', 'Question', 'Response', 'Race', 'Gender', 'Province', 'Occupation', 'Demography']
final_data = merged_data[final_columns]

# Save the final merged file
merged_file_path = r'C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\Final_Merged_Categorized_Survey.xlsx'
final_data.to_excel(merged_file_path, index=False)

print("Merge completed: Final data saved to:", merged_file_path)


Merge completed: Final data saved to: C:\Users\Gebruiker\Desktop\My Lab\Wildfire-Assessment\Processeddata\Final_Merged_Categorized_Survey.xlsx
